In [2]:
import networkx as nx
import numpy as np

In [3]:
def generate_pairs(N):
    pairs = []
    for i in range(N):
        for j in range(i + 1,N):
            pair = (i,j)
            pairs.append(pair)
    return pairs

In [4]:
def random_graph(N,p):

    G = nx.Graph()
    G.add_nodes_from(np.arange(N))
    pairs = generate_pairs(N)
    for pair in pairs:
        if np.random.uniform(0,1) < p:
            G.add_edge(pair[0],pair[1])
    return G

In [32]:
def watts_strogatz(N,K,p):
    
    G = nx.Graph()
    nodes = np.arange(1,N + 1)
    neigbors_index = np.arange(1,K//2 + 1)
    G.add_nodes_from(nodes)
    
    for i in range(len(nodes)):
        for ind in neigbors_index:
            G.add_edge(nodes[i],nodes[i - ind])
    return G